# HugeGraph

>[HugeGraph](https://hugegraph.apache.org/) 是一个便捷、高效且适应性强的图数据库，与 `Apache TinkerPop3` 框架以及 `Gremlin` 查询语言兼容。
>
>[Gremlin](https://en.wikipedia.org/wiki/Gremlin_(query_language)) 是由 `Apache Software Foundation` 的 `Apache TinkerPop` 开发的图遍历语言和虚拟机。

本笔记本展示了如何使用 LLM 为 [HugeGraph](https://hugegraph.apache.org/cn/) 数据库提供自然语言接口。

## 设置

你需要有一个正在运行的 HugeGraph 实例。
你可以通过运行以下脚本来启动一个本地 docker 容器：

```
docker run \
    --name=graph \
    -itd \
    -p 8080:8080 \
    hugegraph/hugegraph
```

如果想在应用程序中连接 HugeGraph，需要安装 python sdk：

```
pip3 install hugegraph-python
```

如果你正在使用 docker 容器，你需要等待几秒钟让数据库启动，然后我们需要为数据库创建 schema 和写入图数据。

In [13]:
from hugegraph.connection import PyHugeGraph

client = PyHugeGraph("localhost", "8080", user="admin", pwd="admin", graph="hugegraph")

首先，我们为简单的电影数据库创建模式：

In [4]:
"""schema"""
schema = client.schema()
schema.propertyKey("name").asText().ifNotExist().create()
schema.propertyKey("birthDate").asText().ifNotExist().create()
schema.vertexLabel("Person").properties(
    "name", "birthDate"
).usePrimaryKeyId().primaryKeys("name").ifNotExist().create()
schema.vertexLabel("Movie").properties("name").usePrimaryKeyId().primaryKeys(
    "name"
).ifNotExist().create()
schema.edgeLabel("ActedIn").sourceLabel("Person").targetLabel(
    "Movie"
).ifNotExist().create()

'create EdgeLabel success, Detail: "b\'{"id":1,"name":"ActedIn","source_label":"Person","target_label":"Movie","frequency":"SINGLE","sort_keys":[],"nullable_keys":[],"index_labels":[],"properties":[],"status":"CREATED","ttl":0,"enable_label_index":true,"user_data":{"~create_time":"2023-07-04 10:48:47.908"}}\'"'

然后我们可以插入一些数据。

In [26]:
"""graph"""
g = client.graph()
g.addVertex("Person", {"name": "Al Pacino", "birthDate": "1940-04-25"})
g.addVertex("Person", {"name": "Robert De Niro", "birthDate": "1943-08-17"})
g.addVertex("Movie", {"name": "The Godfather"})
g.addVertex("Movie", {"name": "The Godfather Part II"})
g.addVertex("Movie", {"name": "The Godfather Coda The Death of Michael Corleone"})

g.addEdge("ActedIn", "1:Al Pacino", "2:The Godfather", {})
g.addEdge("ActedIn", "1:Al Pacino", "2:The Godfather Part II", {})
g.addEdge(
    "ActedIn", "1:Al Pacino", "2:The Godfather Coda The Death of Michael Corleone", {}
)
g.addEdge("ActedIn", "1:Robert De Niro", "2:The Godfather Part II", {})

1:Robert De Niro--ActedIn-->2:The Godfather Part II

## 创建 `HugeGraphQAChain`

我们现在可以创建 `HugeGraph` 和 `HugeGraphQAChain`。要创建 `HugeGraph`，我们只需将数据库对象传递给 `HugeGraph` 构造函数即可。

In [27]:
from langchain.chains import HugeGraphQAChain
from langchain_community.graphs import HugeGraph
from langchain_openai import ChatOpenAI

In [28]:
graph = HugeGraph(
    username="admin",
    password="admin",
    address="localhost",
    port=8080,
    graph="hugegraph",
)

## 刷新图架构信息

如果数据库的架构发生变化，您可以刷新生成 Gremlin 语句所需的架构信息。

In [29]:
# graph.refresh_schema()

In [30]:
print(graph.get_schema)

Node properties: [name: Person, primary_keys: ['name'], properties: ['name', 'birthDate'], name: Movie, primary_keys: ['name'], properties: ['name']]
Edge properties: [name: ActedIn, properties: []]
Relationships: ['Person--ActedIn-->Movie']



## 查询图

现在我们可以使用图 Gremlin QA 链向图表提出问题

In [31]:
chain = HugeGraphQAChain.from_llm(ChatOpenAI(temperature=0), graph=graph, verbose=True)

In [32]:
chain.run("Who played in The Godfather?")



> Entering new  chain...
Generated gremlin:
g.V().has('Movie', 'name', 'The Godfather').in('ActedIn').valueMap(true)
Full Context:
[{'id': '1:Al Pacino', 'label': 'Person', 'name': ['Al Pacino'], 'birthDate': ['1940-04-25']}]

> Finished chain.


'Al Pacino played in The Godfather.'